In [79]:
from envs.gridworld_contextual import TaskSettableGridworld
from gymnasium.wrappers import TimeLimit
from envs.contextual_env import CtxDictWrapper, ctx_visibility, exp_group
import numpy as np
from ray.rllib.policy.policy import Policy
from distral.distral_ppo_torch_model import DistralCentralTorchModel, DistralTorchModel
from ray.rllib.models import ModelCatalog

from gymnasium.wrappers import  RecordVideo, RecordEpisodeStatistics
ModelCatalog.register_custom_model(
    "central",
    DistralCentralTorchModel,
)
import os
ModelCatalog.register_custom_model(
    "local",
    DistralTorchModel,
)
# from gymnasium.wrappers import

In [80]:
max_steps = 32
ctx_mode = 0
env_creator = lambda config: CtxDictWrapper(RecordVideo(RecordEpisodeStatistics(TimeLimit(TaskSettableGridworld(config), max_episode_steps=max_steps ), deque_size = 128),video_folder= './results/recordings/', episode_trigger=lambda x:x%16 ==0, video_length=32),
                                            key = 'region', ctx_visible= ctx_mode
                                         )
learner_0 = Policy.from_checkpoint('./results/GridWorld/V10.1.5/Set0/ctx_hid/baseline/default/PPO_2x64/PPO_MADnCEnv_b16be_00000_0_grad_clip=100,seed=0_2024-01-19_11-06-24/checkpoint_000000/policies/learner_0')
config = {'size': (7, 5),
                            'corridor': (1, 1),
                            'region': {0: (1, 1), 1: (-2, -2)}}

env = env_creator(config)

/home/mohsen/anaconda3/envs/ray2.2/lib/python3.8/site-packages/gymnasium/wrappers/record_video.py:87: UserWarning: WARN: Overwriting existing videos at /home/mohsen/Documents/Research/ray2.9/results/recordings folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


In [95]:
print(env.episode_returns)

[0.]


In [97]:
success =0
rewards = []
for i in range(8):
    obs = env.reset()
    done = False

    while not done:
        # print(obs)
        action = learner_0.compute_single_action(obs[0])
        obs = env.step(action[0])
        done = obs[2] or obs[3]
        if obs[2]:
            success+=1
    # print(i, obs)
    rewards.append(env.episode_returns)
print(np.mean(env.return_queue), rewards, success/128)

-3.2546873 [array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32)] 0.0


In [100]:
x = np.array(env.return_queue).reshape(-1)
print(len(x))

128


In [39]:
learner_0.model.distill_out()
dist = learner_0.dist_class(learner_0.model.distill_out())

In [41]:
class ExperimentEvaluation():
    def __init__(self, experiment_dir ):
        self.log_dir = experiment_dir
        self.experiments = None


    def get_experiments(self):
        #         name_map= {}
        #         param_map ={}
        # baselines ={}
        results = {}
        files_list = list()
        experiment_sets = list()
        for (dirpath, dirnames, filenames) in os.walk(self.log_dir):
            files_list += [os.path.join(dirpath, file) for file in filenames if ('result.json' in file

                                                                                 and not os.path.exists(
                        os.path.join(dirpath, 'error.txt')))]

        for f in files_list:
            splts = f.split('/')
            setup = splts[-2]
            name = splts[-3]
            curriculum = splts[-4]
            learner_group = splts[-5]
            visibility= splts[-6]
            experiment_set = splts[-7]
            parameters = '_'.join(setup.split('_')[5:-2])
            # seed = int(re.search('seed=(.\d+?),', parameters).group(1))
            #             print(f)
            seed = int(parameters.split('seed=')[1])
            parameters = ','.join(parameters.split(',')[:-1])
            # parameters = ','.join([splt[0], ] + splt[1].split(',')[1:])
                # print(parameters)
            # parameters = parameters.split('min_sample')[0]
            if experiment_set not in experiment_sets:
                experiment_sets.append(experiment_set)
            if parameters:
                if parameters[-1] == ',':
                    parameters = parameters[:-1]

                parameters = parameters.replace('distral_', '')
                parameters = parameters.replace(',,', ',')
            experiment_id = ( experiment_set, visibility,learner_group, curriculum, name, parameters)
            # print(experiment_id)
            if experiment_id in results:
                # if parameters in results[name]['config']:
                results[experiment_id]['files'].append(f)
                results[experiment_id]['seeds'].append(seed)

            else:
                results[experiment_id] = {'files': [f, ], 'seeds': [seed, ], 'exp_dir': }
                # results[name]['config'].append(parameters)
            #                 results[name]['learner'] =
            #                 results[name]['curriculum'] = name.split('_')[1]
            #                 results[name]['context_visible'] = 'ctxvis' in name

            # else:
            #     results[name] = {parameters: {'files': [f, ], 'seeds': [seed, ]},
            #                      'config': [parameters, ],
            #                      'learner': name.split('_')[0],
            #                      'curriculum': name.split('_')[1],
            #                      'context_visible': 'ctxvis' in name}

        self.experiment_sets = experiment_sets
        return results

In [48]:
dir = './results/GridWorld/V10.1.5'

ev = ExperimentEvaluation(experiment_dir=dir)

In [102]:

trial_dir = './results/GridWorld/V10.1.5/Set0/ctx_hid/baseline/default/PPO_Central_2x64/PPO_MADnCEnv_54171_00009_9_grad_clip=100,seed=9_2024-01-19_12-08-13/'
chk = next(os.walk(trial_dir))[1]
chk.sort(reverse= True)
policies = next(os.walk(os.path.join(trial_dir, chk[0],'policies')))[1]
policies.sort()

print(chk)

# for (path_, dirs, fnames ) in os.walk(dir):
#
#     if 'checkpoint' in path_:
#         print( path_)



['checkpoint_000004', 'checkpoint_000003', 'checkpoint_000002', 'checkpoint_000001', 'checkpoint_000000']


In [67]:
import json
with open(os.path.join(trial_dir, 'params.json'), 'r') as f:
    data = json.load(f)['env_config']

In [68]:
data

{'agent_config': [{'corridor': [1, 1],
   'region': {'0': [1, 1], '1': [-2, -2]},
   'size': [7, 5]},
  {'corridor': [1, 1], 'region': {'0': [1, 1], '1': [-2, -2]}, 'size': [7, 5]},
  {'corridor': [1, 1], 'region': {'0': [1, 1], '1': [3, 1]}, 'size': [7, 5]},
  {'corridor': [1, 1], 'region': {'0': [3, 1], '1': [3, 3]}, 'size': [7, 5]},
  {'corridor': [1, 1], 'region': {'0': [3, 3], '1': [5, 3]}, 'size': [7, 5]}],
 'non_training_envs': 1,
 'num_agents': 5}